# Training models in Azure Databricks and deploying them on Azure ML

This notebook demostrates how to train models in Azure Databricks (or any Databricks implementation) and deploying those models on Azure ML. Two workflows are demostrated here depending on the level of integration you want to keep and how you want to do tracking:

1. **Scenario 1: Training on Azure Databricks while tracking experiments and models in Azure ML:** This example shows how to do training of models in Azure Databricks while doing all the tracking of experiments in Azure ML (instead of in the MLflow instance running on Azure Databricks). This will also allow you to seemessly deploy models to Azure ML deployment targets in the easiest way.
2. **Scenario 2: Training and tracking experiments in Azure Databricks with Model Registries in Azure ML:** This example shows how to do training and tracking of models in Azure Databricks. Tracking of experiments happens here in the MLflow instance running on Azure Databricks. However, model registries are kept on Azure ML to allow quick model's deployment from a centralized location and registry of models.

Read each scenario to know more about advantages and disadvantages of each approach.

## Before starting

To run this notebook ensure you have:
- A Databricks workspace with a compute with the following libraries:
  - xgboost
  - scikit-learn==1.1.1
  - pandas
  - numpy
  - mlflow
  - azureml-mlflow

Also, configure the following variables:

You will need to connect MLflow to the Azure Machine Learning workspace you want to work on. MLflow uses the tracking URI to indicate the MLflow server you want to connect to. There are multiple ways to get the Azure Machine Learning MLflow Tracking URI. In this tutorial we will use the Azure ML SDK for Python, but you can check [Set up tracking environment - Azure Machine Learning Docs](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-use-mlflow-cli-runs#set-up-tracking-environment) for more alternatives. First of all, test whether Azure ML endpoints resolves to an private IP address

In [0]:
%sh

nslookup <<Azure ML guid>>.workspace.westeurope.api.azureml.ms

Server:		168.63.129.16
Address:	168.63.129.16#53

Non-authoritative answer:
e2185a76-1fa3-4ad6-a882-f5d6ce95bdb5.workspace.westeurope.api.azureml.ms	canonical name = e2185a76-1fa3-4ad6-a882-f5d6ce95bdb5.workspace.westeurope.privatelink.api.azureml.ms.
Name:	e2185a76-1fa3-4ad6-a882-f5d6ce95bdb5.workspace.westeurope.privatelink.api.azureml.ms
Address: 10.0.0.4



You can use the workspace object to get the tracking URI:

In [0]:
#
# See https://learn.microsoft.com/en-us/azure/machine-learning/how-to-use-mlflow-azure-databricks?tabs=cli%2Cmlflow#tracking-exclusively-on-azure-machine-learning-workspace how to get the tracking URL. In the git repo https://github.com/rebremer/databricks-mlflow-azureml-moe-vnet an Azure CLI command is used
#

azureml_tracking_uri = "azureml://<<Azure ML guid>>.workspace.westeurope.api.azureml.ms/mlflow/v1.0/subscriptions/<<your subscription>>/resourceGroups/<<your resource group>>/providers/Microsoft.MachineLearningServices/workspaces/<<your azure ml workspace name>>"

## Scenario 1: Training on Azure Databricks while tracking experiments and models in Azure ML

In this scenario, you want tracking and model registry to happen in Azure Machine Learning, however, you want to keep training models in Azure Databricks. To do that, we need to configure the tracking URI on each instance of Databricks:

In [0]:
import mlflow

mlflow.set_tracking_uri(azureml_tracking_uri)

### Training a heart condition classifier

#### Configuring the experiment

Tracking of experiments will happen in Azure ML and hence we need to use the naming convention we generally use with MLflow. 

>Note that naming in Azure Databricks is different as you have to use the path to where the experiment will be saved. In Azure ML and in general MLflow this is not the case.

In [0]:
mlflow.set_experiment(experiment_name="test2-heart-condition-classifier")

InteractiveBrowserCredential.get_token failed: Failed to open a browser
2023/01/18 14:56:55 INFO mlflow.tracking.fluent: Experiment with name 'test2-heart-condition-classifier' does not exist. Creating a new experiment.
InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
Out[8]: <Experiment: artifact_location='', creation_time=1674053815727, experiment_id='a67508a9-e760-4d86-bbdc-6ccaf3ab76a9', last_update_time=None, lifecycle_stage='active', name='test2-heart-condition-classifier', tags={}>

> **About authentication:** Interactive Authentication or Device Authentication will be triggered when you can `set_experiment`. This is used to authenticate against Azure Machine Learning and be able to call the tracking API. If you are executing the code in the context of a job where interactive authentication is not possible, see the example `notebooks/using-mlflow/train-with-mlflow/xgboost_service_principal.ipynb` for an example about how to use a Service Principal to authenticate against Azure Machine Learning and MLflow.

Since all the tracking is happening in Azure ML, you can train and register models in the regular way you do with mlflow.

#### Exploring the data

In [0]:
import pandas as pd

file_url = "https://azuremlexampledata.blob.core.windows.net/data/heart-disease-uci/data/heart.csv"
df = pd.read_csv(file_url)
display(df)

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0
56,1,2,120,236,0,0,178,0,0.8,1,0,normal,0
62,0,4,140,268,0,2,160,0,3.6,3,2,normal,1
57,0,4,120,354,0,0,163,1,0.6,1,0,normal,0
63,1,4,130,254,0,2,147,0,1.4,2,1,reversible,1
53,1,4,140,203,1,2,155,1,3.1,3,0,reversible,0


As we can see, some of the variables are categorical. To make it simpler for our model to handle these values, let's use their encoded values:

In [0]:
df["thal"] = df["thal"].astype("category").cat.codes

Let's split our dataset in train and test, so we can assess the performance of the model without overfitting the dataset

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.drop("target", axis=1), df["target"], test_size=0.3
)

#### Training a model

We are going to use autologging capabilities in MLflow to track parameters and metrics:

In [0]:
mlflow.xgboost.autolog()

Let's create a simple classifier and train it:

In [0]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, recall_score

model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")

/databricks/.python_edge_libs/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [0]:
with mlflow.start_run() as run:
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    print("Recall: %.2f%%" % (recall * 100.0))

InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
2023/01/18 14:58:39 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/databricks/python/lib/python3.9/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes 

### Registering the model in Azure ML

Since our experiments are being tracked in Azure ML, we can simply register models in the registry like this:

In [0]:
mlflow.register_model(
    model_uri=f"runs:/{run.info.run_id}/model", name="test2-databricks-heart-classifier"
)

InteractiveBrowserCredential.get_token failed: Failed to open a browser
Successfully registered model 'test2-databricks-heart-classifier'.
InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
2023/01/18 15:10:25 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: test2-databricks-heart-classifier, version 1
Created version '1' of model 'test2-databricks-heart-classifier'.
Out[15]: <ModelVersion: creation_timestamp=1674054625718, current_stage='None', description='', last_updated_timestamp=1674054625718, name='test2-databricks-heart-classifier', run_id='b9f16546-08e0-4fe8-8774-c437d010fe61', run_link='', source='azureml://e2185a76-1fa3-4ad6-a882-f5d6ce95bdb5.workspace.westeurope.api.azureml.ms/mlflow/v2.0/subscriptions/a800eb7d-7307-4c88-9de6-44467ece3a56/resourceGroups/test-dbramlmoe-rg/providers/Microsoft.Ma